In [1]:
import numpy as np
import skallel_stats
from skallel_stats import pairwise_distance
#skallel_stats.__version__

## Smaller dataset (20MB)

In [2]:
# simulate some genotype data
x = np.random.choice(np.array([0, 1, 2], dtype='i1'), 
                     p=[.7, .2, .1,], 
                     size=(20_000, 1_000))

In [3]:
x.nbytes / 1e6

20.0

### Single CPU core

In [4]:
# warm-up jit
pairwise_distance(x, metric='cityblock')

array([11931., 11996., 12124., ..., 11914., 11858., 11926.])

In [5]:
%%time
pairwise_distance(x, metric='cityblock')

CPU times: user 7.19 s, sys: 72.5 ms, total: 7.26 s
Wall time: 7.27 s


array([11931., 11996., 12124., ..., 11914., 11858., 11926.])

### Multiple CPU cores via Dask

In [6]:
import dask.array as da
x_dask = da.from_array(x, chunks=(2_000, None))
x_dask

dask.array<array, shape=(20000, 1000), dtype=int8, chunksize=(2000, 1000)>

In [7]:
# warm-up jit
pairwise_distance(x_dask, metric='cityblock').compute()

array([11931., 11996., 12124., ..., 11914., 11858., 11926.])

In [8]:
%%time
pairwise_distance(x_dask, metric='cityblock').compute()

CPU times: user 20.4 s, sys: 40.4 ms, total: 20.4 s
Wall time: 3.18 s


array([11931., 11996., 12124., ..., 11914., 11858., 11926.])

### Single GPU

In [9]:
!nvidia-smi | head

Mon Jul 29 21:18:13 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro M1000M       Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0    N/A /  N/A |   1083MiB /  2004MiB |     15%      Default |
+-------------------------------+----------------------+----------------------+


In [10]:
from numba import cuda

In [11]:
%%time
x_cuda = cuda.to_device(x)
x_cuda

CPU times: user 18.2 ms, sys: 31.9 ms, total: 50.2 ms
Wall time: 49.5 ms


In [12]:
# warm-up jit
pairwise_distance(x_cuda, metric='cityblock')
cuda.synchronize()

In [13]:
%%time
dist_cuda = pairwise_distance(x_cuda, metric='cityblock')
cuda.synchronize()

CPU times: user 431 ms, sys: 176 ms, total: 607 ms
Wall time: 605 ms


In [14]:
%time dist_cuda.copy_to_host()

CPU times: user 2.13 ms, sys: 0 ns, total: 2.13 ms
Wall time: 1.26 ms


array([11931., 11996., 12124., ..., 11914., 11858., 11926.], dtype=float32)

In [16]:
%%time
dist_cuda = pairwise_distance(x_cuda, metric='euclidean')
cuda.synchronize()

CPU times: user 439 ms, sys: 220 ms, total: 659 ms
Wall time: 657 ms


In [18]:
%%time
dist_cuda = pairwise_distance(x_cuda, metric='hamming')
cuda.synchronize()

CPU times: user 501 ms, sys: 156 ms, total: 657 ms
Wall time: 655 ms


In [26]:
%%time
dist_cuda = pairwise_distance(x_cuda, metric='jaccard')
cuda.synchronize()

CPU times: user 506 ms, sys: 156 ms, total: 662 ms
Wall time: 658 ms


## Larger dataset (1GB) 

In [19]:
# x_big = da.random.choice(
#     np.array([0, 1, 2], dtype='i1'), 
#     p=[.7, .2, .1], 
#     size=(1_000_000, 1_000),
#     chunks=(50_000, None))
# x_big

In [20]:
# x_big.to_zarr('example.zarr', component='x_big', overwrite=True)

In [21]:
import zarr
x_big_zarr = zarr.open('example.zarr')['x_big']
x_big_zarr.info

Name,/x_big
Type,zarr.core.Array
Data type,int8
Shape,"(1000000, 1000)"
Chunk shape,"(50000, 1000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,1000000000 (953.7M)
No. bytes stored,571540566 (545.1M)


In [22]:
x_big_dask = da.from_array(x_big_zarr)
x_big_dask

dask.array<array, shape=(1000000, 1000), dtype=int8, chunksize=(100000, 1000)>

### Multiple CPUs via Dask

In [30]:
%%time
dist_big = pairwise_distance(x_big_dask, metric='cityblock').compute()
dist_big

CPU times: user 23min 12s, sys: 7.55 s, total: 23min 20s
Wall time: 3min 29s


array([600315., 599954., 599561., ..., 599505., 599930., 599789.])

### Single GPU via Dask

In [23]:
x_big_dask_cuda = x_big_dask.map_blocks(cuda.to_device)

In [29]:
%%time
dist_big_cuda = pairwise_distance(x_big_dask_cuda, metric='cityblock').compute(num_workers=1)
dist_big_cuda

CPU times: user 23.6 s, sys: 8.01 s, total: 31.6 s
Wall time: 31.6 s


array([600315., 599954., 599561., ..., 599505., 599930., 599789.])

In [31]:
%%time
dist_big_cuda = pairwise_distance(x_big_dask_cuda, metric='cityblock').compute(num_workers=2)
dist_big_cuda

CPU times: user 23.5 s, sys: 8.34 s, total: 31.9 s
Wall time: 30.8 s


array([600315., 599954., 599561., ..., 599505., 599930., 599789.])

In [32]:
%%time
dist_big_cuda = pairwise_distance(x_big_dask_cuda, metric='euclidean').compute(num_workers=1)
dist_big_cuda

CPU times: user 25.6 s, sys: 8.71 s, total: 34.3 s
Wall time: 34.3 s


array([937.96428503, 937.9594874 , 937.53026618, ..., 937.3339853 ,
       938.4881459 , 937.75209944])

In [28]:
%%time
dist_big_cuda = pairwise_distance(x_big_dask_cuda, metric='hamming').compute(num_workers=1)
dist_big_cuda

CPU times: user 25.1 s, sys: 8.81 s, total: 34 s
Wall time: 33.9 s


array([0.460584, 0.460047, 0.45986 , ..., 0.45996 , 0.459515, 0.459994])

In [24]:
%%time
dist_big_cuda = pairwise_distance(x_big_dask_cuda, metric='jaccard').compute(num_workers=1)
dist_big_cuda

CPU times: user 25.6 s, sys: 9.01 s, total: 34.6 s
Wall time: 34.6 s


array([0.90127329, 0.90228824, 0.90150774, ..., 0.90216892, 0.90146584,
       0.90203569])